In [5]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

In [2]:
from companies.models import Company

In [4]:
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [5]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [6]:
company = Company.objects.create(name="Google", 
                                 ticker=Upper(Value("goog")), 
                                 num_employees=10, 
                                 num_tables=20, 
                                 num_chairs=20
)

In [7]:
companies = Company.objects.all().annotate(name_upper=Upper('name'))

In [8]:
for com in companies:
    print(f"Name: {com.name}, Name Upper: {com.name_upper}")

Name: Company AAA, Name Upper: COMPANY AAA
Name: Company BBB, Name Upper: COMPANY BBB
Name: Company CCC, Name Upper: COMPANY CCC
Name: Google, Name Upper: GOOGLE


In [9]:
print(companies.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", UPPER("companies_company"."name") AS "name_upper" FROM "companies_company"


In [16]:
companies = (
    Company.objects.filter(num_employees__gt=F("num_chairs"))
    .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
    .first()
    )

In [17]:
for com in companies:
    print(F"Name: {com.name}, Chairs needed: {com.chairs_needed}")

TypeError: 'Company' object is not iterable

In [18]:
Company.objects.order_by(Length("name").asc())

<QuerySet [<Company: Google>, <Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>]>

In [19]:
Company.objects.order_by(Length("name").desc())

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

In [ ]:
companies = Company.objects.annotate(
need_chairs = 
)

In [6]:
from books.models import Book

In [7]:
from django.db.models import Avg, Sum, Count, Min, Max
Book.objects.aggregate(sum_price=Sum('price'))

{'sum_price': Decimal('572.41')}

In [8]:
Book.objects.aggregate(avg_rate=Avg('rating'))

{'avg_rate': 4.277966101694915}

In [12]:
Book.objects.filter(publisher__name='Penguin Books').aggregate(avg_publisher=Avg('rating'))

{'avg_publisher': 4.340000000000001}

In [13]:
Book.objects.filter(publisher__name='Oxford University Press').aggregate(avg_publisher=Avg('rating'))

{'avg_publisher': 4.246153846153847}

In [17]:
Publisher.objects.annotate(num_books=Count("book")).values()

<QuerySet [{'id': 1, 'name': 'Penguin Books', 'num_books': 20}, {'id': 2, 'name': 'Oxford University Press', 'num_books': 39}]>

In [20]:
Publisher.objects.annotate(num_book=Avg('book__rating')).values()

<QuerySet [{'id': 1, 'name': 'Penguin Books', 'num_book': 4.340000000000001}, {'id': 2, 'name': 'Oxford University Press', 'num_book': 4.246153846153847}]>

In [22]:
from books.models import Author
Author.objects.annotate(avg_rating=Avg('book__rating')).filter(avg_rating__gt=4.2).values()

<QuerySet [{'id': 22, 'name': 'Joseph Heller', 'age': 76, 'avg_rating': 4.3}, {'id': 4, 'name': 'Jane Austen', 'age': 41, 'avg_rating': 4.4}, {'id': 6, 'name': 'Herman Melville', 'age': 72, 'avg_rating': 4.4}, {'id': 2, 'name': 'George Orwell', 'age': 46, 'avg_rating': 4.425}, {'id': 7, 'name': 'Leo Tolstoy', 'age': 82, 'avg_rating': 4.4}, {'id': 20, 'name': 'Emily Brontë', 'age': 30, 'avg_rating': 4.333333333333333}, {'id': 1, 'name': 'F. Scott Fitzgerald', 'age': 44, 'avg_rating': 4.3999999999999995}, {'id': 18, 'name': 'Ray Bradbury', 'age': 91, 'avg_rating': 4.5}, {'id': 11, 'name': 'Gabriel García Márquez', 'age': 87, 'avg_rating': 4.25}, {'id': 9, 'name': 'Fyodor Dostoevsky', 'age': 59, 'avg_rating': 4.34}, {'id': 16, 'name': 'Mary Shelley', 'age': 53, 'avg_rating': 4.5}, {'id': 12, 'name': 'Aldous Huxley', 'age': 69, 'avg_rating': 4.4}]>

In [26]:
Book.objects.filter(name__startswith='The', publisher__name='Penguin Books').values()

<QuerySet [{'id': 1, 'name': 'The Great Gatsby', 'pages': 180, 'price': Decimal('10.99'), 'rating': 4.3, 'publisher_id': 1, 'pubdate': datetime.date(1925, 4, 10)}, {'id': 5, 'name': 'The Catcher in the Rye', 'pages': 277, 'price': Decimal('6.99'), 'rating': 3.9, 'publisher_id': 1, 'pubdate': datetime.date(1951, 7, 16)}, {'id': 8, 'name': 'The Odyssey', 'pages': 541, 'price': Decimal('9.99'), 'rating': 4.7, 'publisher_id': 1, 'pubdate': datetime.date(800, 12, 1)}, {'id': 14, 'name': 'The Hobbit', 'pages': 310, 'price': Decimal('10.49'), 'rating': 4.7, 'publisher_id': 1, 'pubdate': datetime.date(1937, 9, 21)}, {'id': 17, 'name': 'The Hitchhiker Guide to the Galaxy', 'pages': 224, 'price': Decimal('6.99'), 'rating': 4.5, 'publisher_id': 1, 'pubdate': datetime.date(1979, 10, 12)}]>

In [27]:
a1 = Author.objects.get(pk=1)
a2 = Author.objects.get(pk=2)

book = Book.objects.get(pk=10)

In [28]:
book.authors.add(a2)

In [29]:
book.authors.all()

<QuerySet [<Author: Author object (2)>, <Author: Author object (5)>]>

In [30]:
book.authors.count()

2

In [31]:
a1.book_set.add(book)

In [32]:
a1.book_set.all()

<QuerySet [<Book: Book object (2)>, <Book: Book object (10)>, <Book: Book object (21)>, <Book: Book object (31)>, <Book: Book object (51)>]>

In [33]:
a1.book_set.remove(book)

In [34]:
a1.book_set.all()

<QuerySet [<Book: Book object (2)>, <Book: Book object (21)>, <Book: Book object (31)>, <Book: Book object (51)>]>